# Anova study on Shapley explanation maps 

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import xlsxwriter
from matplotlib import pyplot as plt

import scipy.stats as stats
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [ ]:
upper_face_aus = ['AU01','AU02','AU04','AU05','AU06','AU07','AU09','AU45']
lower_face_aus = ['AU10','AU12','AU14','AU15','AU17','AU20','AU23','AU25','AU26']

df = pd.read_csv('../dataset/pickled_datasets/output_attribution/shapley_sampling_all_dataset.csv')

In [ ]:
def truncate(n):
    return int(n * 100) / 100

def query_df(df,subject=None,study=None,paradigm=None,trial=None,timestep=None,label=None):
    """
    Function to query a dataframe to very granular level. Expects lists as inputs.
    """
    query = []
    if subject!=None:
        query.append("subject in " + str(subject))
    if study!=None:
        query.append("study in " + str(study))
    if paradigm!=None:
        query.append("paradigm in " + str(paradigm))
    if trial!=None:
        query.append("trial in " + str(trial))
    if timestep!=None:
        query.append("timestep in " + str(timestep))
    if label!=None:
        query.append("label in " + str(label))
    query = " & ".join(query)
    return(df.query(query))

def run_anova(subjects, columns, aus='both', timestep_range=0, showgraph=False):
    
    
    # Choose a timestep.
    if timestep_range==0:
        timestep = list(range(0,87))
    elif timestep_range==1:
        timestep = list(range(0,29))
    elif timestep_range==2:
        timestep = list(range(29,58))
    elif timestep_range==3:
        timestep = list(range(58,87))
    else:
        raise ValueError("Wrong timestep range")
        
    cw_paradigms = ['cw1','cw2','caw1','caw2']
    wg_paradigms = ['wg1','wg2','wag1','wag2']
    
    dfq = query_df(df, label=[0,1], subject=subjects, timestep=timestep, paradigm=wg_paradigms)

    # Fix the columns to consider, group data, and replace combine paradigms if chosen.
    cols_consider = columns
    dfin_anova = dfq.groupby(cols_consider).mean()[upper_face_aus+lower_face_aus].reset_index()
#     dfin_anova = dfin_anova.replace('cw1', 'cw')
#     dfin_anova = dfin_anova.replace('cw2', 'cw')
#     dfin_anova = dfin_anova.replace('caw1', 'caw')
#     dfin_anova = dfin_anova.replace('caw2', 'caw')
#     dfin_anova = dfin_anova.replace('wg1', 'wg')
#     dfin_anova = dfin_anova.replace('wg2', 'wg')
#     dfin_anova = dfin_anova.replace('wag1', 'wag')
#     dfin_anova = dfin_anova.replace('wag2', 'wag')

    dfin_anova = dfin_anova.replace('cw1', 'cw')
    dfin_anova = dfin_anova.replace('cw2', 'cw')
    dfin_anova = dfin_anova.replace('caw1', 'cw')
    dfin_anova = dfin_anova.replace('caw2', 'cw')
    dfin_anova = dfin_anova.replace('wg1', 'wg')
    dfin_anova = dfin_anova.replace('wg2', 'wg')
    dfin_anova = dfin_anova.replace('wag1', 'wg')
    dfin_anova = dfin_anova.replace('wag2', 'wg')

    # Select the AUs to study.
    if aus=='Both':
        aus_to_study = upper_face_aus + lower_face_aus
    elif aus=='Upper':
        aus_to_study = upper_face_aus
    elif aus=='Lower':
        aus_to_study = lower_face_aus
    elif aus in upper_face_aus + lower_face_aus:
        aus_to_study = [aus]
    else:
        raise ValueError("AU choices are 'both', 'upper', 'lower', or one in 17 AUs considered for the study. Please choose one among these.")
    
    # Define the columns to extract
    columns_to_extract = cols_consider + aus_to_study
    
    # Melt the columns for ANOVA
    d_melt = pd.melt(dfin_anova[columns_to_extract], id_vars=['label'], value_vars=aus_to_study)
    d_melt.columns = ['label', 'AU', 'value']
    
    # Saving hack to fix issues with pandas
    d_melt.to_csv('meltdf_predictions.csv')
    d_melt = pd.read_csv('meltdf_predictions.csv')
    
    if showgraph:
        plt.figure(figsize=(10,10))
        sns.boxplot(x="label", y="value", hue="AU", data=d_melt, palette="Set3")

    # Ordinary Least Squares (OLS) model
    model = ols('value ~ C(label) + C(AU) + C(label):C(AU)', data=d_melt).fit()
    anova_table = sm.stats.anova_lm(model, typ=1)
    return (anova_table)


global row
global col

row = 0
col = 0


def return_anova_results(anova_results):
    F_label = anova_results['F'][0]
    p_label = anova_results['PR(>F)'][0]
    F_AU = anova_results['F'][1]
    p_AU = anova_results['PR(>F)'][1]
    
    return(F_label, p_label, F_AU, p_AU)

def run_experiments_and_saveresults(subject):
    global row
    global col
    
    if type(subject) != list:
        subject = [subject]
    
    # if we just consider the impact of paradigms
    cols_to_consider = ['paradigm', 'label']
    
    # for aus in ['Both', 'Upper', 'Lower']:
    for aus in upper_face_aus+lower_face_aus:
        
        if type(subject)==list:
            if len(subject)==1:
                worksheet.write(row+1, col, subject[0])
            elif len(subject)==2:
                worksheet.write(row+1, col, 'HSR')
            elif len(subject)==7:
                worksheet.write(row+1, col, 'All')
            else:
                worksheet.write(row+1, col, 'LSR')
        else:
            worksheet.write(row+1, col, subject)
        
        worksheet.write(row+1, col+1, aus)
        
        for i, timestep_range in enumerate(range(4)):
            anova_results = run_anova(subjects=subject, columns=cols_to_consider, aus=aus, timestep_range=timestep_range)
            F_label, p_label, F_AU, p_AU = return_anova_results(anova_results)
            
            if p_label < 0.005:
                worksheet.write(row+1, col+2+i, "F="+str(truncate(F_label))+", p<0.005", cell_format)
            else:
                worksheet.write(row+1, col+2+i, "F="+str(truncate(F_label))+", p>0.005")
        row+=1
        

workbook = xlsxwriter.Workbook('anova_study_table_individual_au.xlsx')
worksheet = workbook.add_worksheet()

cell_format = workbook.add_format({'bold': True, 'font_color': 'red'})
merge_format = workbook.add_format({
    'bold': 1,
    # 'border': 1,
    'align': 'center',
    'valign': 'vcenter'})
    #'fg_color': 'yellow'})

worksheet.set_column('C:F', 18)

worksheet.merge_range('A1:A2', "Subject ID", merge_format)
worksheet.merge_range('B1:B2', "AU location", merge_format)
worksheet.merge_range('C1:F1', "Timestep Ranges", merge_format)


worksheet.write(row+1, col+2, "0-1500 ms")
worksheet.write(row+1, col+3, "0-500 ms")
worksheet.write(row+1, col+4, "500-1000 ms")
worksheet.write(row+1, col+5, "1000-1500ms")

row+=1

subjects = ['971', '982']
run_experiments_and_saveresults(subjects)


subjects = ['942', '970', '1131', '1196', '1214']
run_experiments_and_saveresults(subjects)


subjects = ['942', '970', '971', '982', '1131', '1196', '1214']
run_experiments_and_saveresults(subjects)

workbook.close()

print("Ran experiments and wrote to Excel file")

## New time windows and tighter p-value ranges

In [ ]:
def truncate(n):
    return int(n * 100) / 100

def query_df(df,subject=None,study=None,paradigm=None,trial=None,timestep=None,label=None):
    """
    Function to query a dataframe to very granular level. Expects lists as inputs.
    """
    query = []
    if subject!=None:
        query.append("subject in " + str(subject))
    if study!=None:
        query.append("study in " + str(study))
    if paradigm!=None:
        query.append("paradigm in " + str(paradigm))
    if trial!=None:
        query.append("trial in " + str(trial))
    if timestep!=None:
        query.append("timestep in " + str(timestep))
    if label!=None:
        query.append("label in " + str(label))
    query = " & ".join(query)
    return(df.query(query))

def run_anova(subjects, columns, aus='both', timestep_range=0, showgraph=False):
    
    
    # Choose a timestep.
    if timestep_range==0:
        timestep = list(range(0,87))
    elif timestep_range==1:
        timestep = list(range(0,29))
    elif timestep_range==2:
        timestep = list(range(29,46))
    elif timestep_range==3:
        timestep = list(range(63,87))
    else:
        raise ValueError("Wrong timestep range")
        
    cw_paradigms = ['cw1','cw2','caw1','caw2']
    wg_paradigms = ['wg1','wg2','wag1','wag2']
    
    dfq = query_df(df, label=[0,1], subject=subjects, timestep=timestep, paradigm=wg_paradigms)

    # Fix the columns to consider, group data, and replace combine paradigms if chosen.
    cols_consider = columns
    dfin_anova = dfq.groupby(cols_consider).mean()[upper_face_aus+lower_face_aus].reset_index()
#     dfin_anova = dfin_anova.replace('cw1', 'cw')
#     dfin_anova = dfin_anova.replace('cw2', 'cw')
#     dfin_anova = dfin_anova.replace('caw1', 'caw')
#     dfin_anova = dfin_anova.replace('caw2', 'caw')
#     dfin_anova = dfin_anova.replace('wg1', 'wg')
#     dfin_anova = dfin_anova.replace('wg2', 'wg')
#     dfin_anova = dfin_anova.replace('wag1', 'wag')
#     dfin_anova = dfin_anova.replace('wag2', 'wag')

    dfin_anova = dfin_anova.replace('cw1', 'cw')
    dfin_anova = dfin_anova.replace('cw2', 'cw')
    dfin_anova = dfin_anova.replace('caw1', 'cw')
    dfin_anova = dfin_anova.replace('caw2', 'cw')
    dfin_anova = dfin_anova.replace('wg1', 'wg')
    dfin_anova = dfin_anova.replace('wg2', 'wg')
    dfin_anova = dfin_anova.replace('wag1', 'wg')
    dfin_anova = dfin_anova.replace('wag2', 'wg')

    # Select the AUs to study.
    if aus=='Both':
        aus_to_study = upper_face_aus + lower_face_aus
    elif aus=='Upper':
        aus_to_study = upper_face_aus
    elif aus=='Lower':
        aus_to_study = lower_face_aus
    elif aus in upper_face_aus + lower_face_aus:
        aus_to_study = [aus]
    else:
        raise ValueError("AU choices are 'both', 'upper', 'lower', or one in 17 AUs considered for the study. Please choose one among these.")
    
    # Define the columns to extract
    columns_to_extract = cols_consider + aus_to_study
    
    # Melt the columns for ANOVA
    d_melt = pd.melt(dfin_anova[columns_to_extract], id_vars=['label'], value_vars=aus_to_study)
    d_melt.columns = ['label', 'AU', 'value']
    
    # Saving hack to fix issues with pandas
    d_melt.to_csv('meltdf_predictions.csv')
    d_melt = pd.read_csv('meltdf_predictions.csv')
    
    if showgraph:
        plt.figure(figsize=(10,10))
        sns.boxplot(x="label", y="value", hue="AU", data=d_melt, palette="Set3")

    # Ordinary Least Squares (OLS) model
    model = ols('value ~ C(label) + C(AU) + C(label):C(AU)', data=d_melt).fit()
    anova_table = sm.stats.anova_lm(model, typ=1)
    return (anova_table)


global row
global col

row = 0
col = 0


def return_anova_results(anova_results):
    F_label = anova_results['F'][0]
    p_label = anova_results['PR(>F)'][0]
    F_AU = anova_results['F'][1]
    p_AU = anova_results['PR(>F)'][1]
    
    return(F_label, p_label, F_AU, p_AU)

def run_experiments_and_saveresults(subject):
    global row
    global col
    
    if type(subject) != list:
        subject = [subject]
    
    # if we just consider the impact of paradigms
    cols_to_consider = ['paradigm', 'label']
    
    # for aus in ['Both', 'Upper', 'Lower']:
    for aus in upper_face_aus+lower_face_aus:
        
        if type(subject)==list:
            if len(subject)==1:
                worksheet.write(row+1, col, subject[0])
            elif len(subject)==2:
                worksheet.write(row+1, col, 'HSR')
            elif len(subject)==7:
                worksheet.write(row+1, col, 'All')
            else:
                worksheet.write(row+1, col, 'LSR')
        else:
            worksheet.write(row+1, col, subject)
        
        worksheet.write(row+1, col+1, aus)
        
        for i, timestep_range in enumerate(range(4)):
            anova_results = run_anova(subjects=subject, columns=cols_to_consider, aus=aus, timestep_range=timestep_range)
            F_label, p_label, F_AU, p_AU = return_anova_results(anova_results)
            
            if p_label < 0.001:
                worksheet.write(row+1, col+2+i, "F="+str(truncate(F_label))+", p<0.001", cell_format)
            elif p_label < 0.005:
                worksheet.write(row+1, col+2+i, "F="+str(truncate(F_label))+", p<0.005", cell_format)
            elif p_label < 0.01:
                worksheet.write(row+1, col+2+i, "F="+str(truncate(F_label))+", p<0.01", cell_format)
            elif p_label < 0.05:
                worksheet.write(row+1, col+2+i, "F="+str(truncate(F_label))+", p<0.05", cell_format)
            else:
                worksheet.write(row+1, col+2+i, "F="+str(truncate(F_label))+", p>0.05")
        row+=1
        

workbook = xlsxwriter.Workbook('anova_study_table_individual_au_t2.xlsx')
worksheet = workbook.add_worksheet()

cell_format = workbook.add_format({'bold': True, 'font_color': 'red'})
merge_format = workbook.add_format({
    'bold': 1,
    # 'border': 1,
    'align': 'center',
    'valign': 'vcenter'})
    #'fg_color': 'yellow'})

worksheet.set_column('C:F', 18)

worksheet.merge_range('A1:A2', "Subject ID", merge_format)
worksheet.merge_range('B1:B2', "AU location", merge_format)
worksheet.merge_range('C1:F1', "Timestep Ranges", merge_format)


worksheet.write(row+1, col+2, "0-1500 ms")
worksheet.write(row+1, col+3, "0-500 ms")
worksheet.write(row+1, col+4, "500-800 ms")
worksheet.write(row+1, col+5, "1100-1500ms")

row+=1

subjects = ['971', '982']
run_experiments_and_saveresults(subjects)


subjects = ['942', '970', '1131', '1196', '1214']
run_experiments_and_saveresults(subjects)


subjects = ['942', '970', '971', '982', '1131', '1196', '1214']
run_experiments_and_saveresults(subjects)

workbook.close()

print("Ran experiments and wrote to Excel file")